# BSC-HGP - Assignment 1 

| Course 	          | BSc                  |
|  ------------- :    |:-------------        :|  
| Stage / Year        | 1                     |
| Module 	          | HCI & GUI Programming | 
|Semester             |	1                     |
|Assignment           |	Assignment 01         | 
|Date of Title Issue  |	9th  Oct 2018              |
|Assignment Deadline  |	22nd Oct 2018 23:55        |
|Assignment %         | 10 %                  |
|Assignment Submission| Upload to moodle      |

Author: Alex Cronin


# Lisa Lewitanski- 2982190


## Design Decisions: 



The application is basic, clear and intuitive.  
The interface is reactive, each time a new valid user input is detected, the display automatically update or create the corresponding graph.  
It also react when a user changes the size of the window, by adapting the display of the application.  

Controls are simplified to be the easiest possible, here are the them:  
- 2 ComboBox, one for the from currency code and the other for the to currency code. When the program get a keyboard event on these 2 boxes, it automatically select a currency code which begin by this letter.
- A SpinBox, which allows the user to choose the amount of the convertion.
- 2 Calendars, one for the from date and the other one for the end date, and both of them are initialised to the actual date to help user.

The controls are horizontally aligned but I have placed a lot of space beetween them for the user to have no ambiguity.  
Moreover, I made a distinctive easy patern by placing controls in such a way that user don't get lost. For exemple, controls are in chronological order, from left to right. Indeed, the 'from' controls are at the left of the screen, and the 'to' at the right.  

Each control has a label, to help user understanding its purpose.  
The colors are smooth and sober, thereby the user has a lot of comfort when using the application.  
 
Finally, the graph is displayed in a large size at the bottom of the controls which automatically make the user understand that it is the result obtained with the parameters he changed. The background color is dark to fit in the overall theme. The curves are red, blue and green to be easily seen by the user.



## Solution

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Oct  2 11:43:14 2018

@author: lisa Lewitanski
@student_id : 2982190
"""

import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QDate
from urllib.request import urlretrieve
import zipfile
import pyqtgraph as pg


class Form(QDialog):

    def __init__(self, parent=None):
        """
        Initialisation of the windows, layout date and widget.
        It also get the content of the zip file and the selected dates.

        """
        super().__init__(parent)
        self.list_to_values = []
        self.list_from_values = []

        self.fromComboBox = QComboBox()

        # get the zip content
        self.zip_content = self.download_unzip()
        to_print = True

        # get all currencies for the comboBox
        self.currencies = self.getcolumn(self.zip_content, to_print)
        self.fromComboBox.addItems(self.currencies)

        # Declare a DoubleSpinBox for get the amount to convert
        self.from_amount = QDoubleSpinBox()
        self.from_amount.setRange(0.01, 10000000.00)
        self.from_amount.setValue(1.00)

        # creation of the Calendar
        self.calendar_begin = QCalendarWidget()
        self.calendar_begin.setVisible(True)
        self.calendar_end = QCalendarWidget()
        self.calendar_end.setVisible(True)

        # get default value from the calendar
        self.from_date = self.calendar_begin.selectedDate().toString("yyyy-MM-dd")
        self.to_date = self.calendar_end.selectedDate().toString("yyyy-MM-dd")

        # set a listener to get user input on calendar
        self.calendar_begin.clicked[QDate].connect(self.getDateB)
        self.calendar_end.clicked[QDate].connect(self.getDateE)

        self.toComboBox = QComboBox()
        self.toComboBox.addItems(self.currencies)

        # Create different label to help the user
        self.to_amount = QLabel("1.00")
        self.from_c = QLabel('From currency: ')
        self.to_c = QLabel('To currency: ')
        self.amount_convert = QLabel('Amount to convert: ')
        self.result = QLabel('Result of conversion based on the most recent rates: ')

        # Initialisation of to and from currency
        self.to_currency = self.toComboBox.currentText()
        self.from_currency = self.fromComboBox.currentText()

        # print all widget in the window
        grid = QGridLayout()
        self.rates_plot = pg.PlotWidget()
        grid.addWidget(self.from_c, 1, 0)
        grid.addWidget(self.fromComboBox, 1, 1)
        grid.addWidget(self.to_c, 1, 2)
        grid.addWidget(self.toComboBox, 1, 3)

        grid.addWidget(self.amount_convert, 2, 0)
        grid.addWidget(self.from_amount, 2, 1)
        grid.addWidget(self.result, 2, 2)
        grid.addWidget(self.to_amount, 2, 3)

        grid.addWidget(self.calendar_begin, 3, 0, 1, 2)
        grid.addWidget(self.calendar_end, 3, 2, 1, 3)
        grid.addWidget(self.rates_plot, 4, 0, 1, 4)
        self.setLayout(grid)

        # Set listeners for user inputs on combobox and spinbox
        self.fromComboBox.currentIndexChanged.connect(self.updateUi)
        self.toComboBox.currentIndexChanged.connect(self.updateUi)
        self.from_amount.valueChanged.connect(self.updateUi)
        self.setWindowTitle("Currency Converter")

        self.legend = None

    def updateUi(self):
        """
        This function update the graph when a user changes the inputs.
        It gets the from currency and the to currency. (to_ and from_)
        It gets the begin and end dates, and check that begin is before end, and dates are not same.
        Then, It creates a list which contains the zip contents. (list_content)
        It also creates a list which contains all currencies. (currencies list)

        After that, with the information above, I create a list of the selected currencies values for
        all the date between the begin and the end date.

        I compute the conversion result according to the amount.

        Finally we create the graph.

        :return: Nothing
        """
        try:
            self.to_currency = self.toComboBox.currentText()
            self.from_currency = self.fromComboBox.currentText()

            self.amt = 0

            list_to = self.to_date.split('-')
            list_from = self.from_date.split('-')

            dtFrom = list_from[0] + list_from[1] + list_from[2]
            dtTo = list_to[0] + list_to[1] + list_to[2]
            if dtFrom != dtTo and min(dtFrom, dtTo) == dtFrom:

                list_content = self.create_list(self.zip_content)
                currencies_list = self.getcolumn(self.zip_content)
                if self.prepare_values_for_graph(list_content, currencies_list, self.to_currency,
                                                 self.from_currency) != -1:
                    to_amt = float(self.list_to_values[len(self.list_to_values) - 1])
                    from_amout = float(self.list_from_values[len(self.list_from_values) - 1])
                    self.amt = float(self.from_amount.value())
                    if to_amt > 0:
                        to_amt = (from_amout / to_amt) * self.amt
                    self.to_amount.setText("%.02f" % to_amt)
                    self.dograph()
                else:
                    self.rates_plot.clear()
            else:
                self.to_amount.setText("N/A")
        except Exception as e:
            print(e)

    def getDateB(self, date):
        """

        :param date: QCalendar begin date

        I get the begin date with the following format 'yyyy-MM-dd'.
        I choose this format because after it will be easier to found this date in the content zip.
        After that, I update the UI to set the new selected date.
        """
        try:
            if date:
                self.from_date = date.toString("yyyy-MM-dd")
                self.updateUi()
        except Exception as e:
            print(e)

    def getDateE(self, date):
        """
        :param date: QCalendar end date

        I get the end date with the following format 'yyyy-MM-dd'.
        I choose this format because after it will be easier to found this date in the content zip.
        After that, I update the UI for set the new selected date.

        """
        try:
            if date:
                self.to_date = date.toString("yyyy-MM-dd")
                self.updateUi()
        except Exception as e:
            print(e)

    def download_unzip(self):
        """

        This function retrieve the zip content with a correct url.
        Before returning the zip content, I decode it because when I get it, everything is in bit format.

        :return: The decode zip content
        """
        try:
            # get the content
            self.data = {}
            zip_url = 'https://www.ecb.europa.eu/stats/eurofxref/eurofxref-hist.zip'
            self.my_file, _ = urlretrieve(zip_url)
            zip_file_object = zipfile.ZipFile(self.my_file, 'r')

            first_file = zip_file_object.namelist()[0]
            self.my_file = zip_file_object.open(first_file)
            my_content = self.my_file.read()

            # return the decode content zip
            return my_content.decode()
        except Exception as e:
            print(e)

    def getCurrency(self, content, index, elem_search, tabcolumns):
        """

        :param content: The decode zip content
        :param index: The date's position
        :param elem_search: The currency. (example: AUD)
        :param tabcolumns: The list which contains all currencies. (example: ['AUD', 'USD'])


        First this function get the currency's position in tabcolumns. (pos_elem)
        After, it gets the Date's position in tabcolumns. (pos_Date)
        With this two positions, I can found the currency's value.

        :return: The currency's value if we have the currency's and date's position.
                 Else, I return ''

        """
        pos_elem = tabcolumns.get(elem_search, -1)
        pos_Date = tabcolumns.get('Date', -1)
        if pos_elem != -1 and pos_Date != -1:
            increment = pos_elem - pos_Date
            return content[index + increment]
        return ''

    def getposdate(self, content, date):
        """

        :param content: The decoded zip content
        :param date: The selected date

        :return: The dates's position in the content
        """
        if date in content:
            return content.index(date)
        return -1

    def create_list(self, content):
        """

        :param content: The decoded zip content

        This function creates a list with the decoded zip content.
        When I create the list I remove the \n and I separate each element using the ','

        :return: A list wich contain the decode zip content
        """
        file_handler = content.replace('\n', '').split(',')
        return file_handler

    def getcolumn(self, content, for_currencies=False):
        """

        :param content: The decoded zip's content
        :param for_currencies: Add date in the list

        This function get all the currencies (AUD, USD ..) and creates a list with these values.
        If we want to add 'Date' in the list, for_currencies will be true. Else it will be false.

        :return: The currency list
        """
        file_handler = content.split('\n')
        title_column = {}
        if file_handler:
            first_colum = file_handler[0].split(',')
            count = 0
            title_column = {}
            for name in first_colum:
                if name != '':
                    if for_currencies:
                        if name != 'Date':
                            title_column[name] = count
                    else:
                        title_column[name] = count
                    count += 1
        return title_column

    def prepare_values_for_graph(self, content, currencies_list, to_, from_):
        """

        :param content: The content list
        :param currencies_list: The tab witch contain all currencies code
        :param to_: The currency code to
        :param from_: The currency code from

        Explication of my algorithm:

        - Normally, for this program we need to create the following dictionary:
            {key: currency_code, value: {date1: code_value1 ,
                                        date2: code_value2,
                                        ... }}
            With this dictionary it's easy to found all values.

        - My algorithm:
            Because I already have the content of each cells in a list, I don't need to create these dictionaries.
            For performances purpose, I just need the first cell of the date, and to have the next one,
            I increment by the number of columns.
            For example, if Date is the column number 1 and there is 42 columns:
            the first date is cell:    1+42=  43
            The second date is cell:   43+42= 85
            ...
            Now to have the corresponding values, I follow a similar principle:
            Let's say I want AUD, which is at column 5, so it is 4 cells after the Date (5 - 1)
            So when I get date at cell 43, the AUD value is at 43+4 = 47, the next one is 85+4=89 ...

            Doing this way may be confusing for some, but it allows to run the program with less impact on the memory,
            and even improve the calculation time.
            Explanation:
            Recommended way:
            Having many dictionaries within a dictionary (memory consumption, and calculation time to create),
            and requesting a value corresponding to a key in a dictionary is longer than just requesting a list index
            My way:
            Having a single list, and incrementing an index.


        Then, I save the to and from currency values in two list (list_to_values, list_from_values).
        Doing this way, I just need to give these list to the graph.


        :return: -1 if I not found the begin or the end date in the list content
                 Else, I return 0
        """

        self.list_to_values = []
        self.list_from_values = []

        # get the position in the list of the begin date
        pos_begin = self.getposdate(content, self.to_date)

        # get the position in the list of the end date
        pos_end = self.getposdate(content, self.from_date)

        # if the position of one date it's not found, it's a problem, so I return an error
        if pos_begin == -1 or pos_end == -1:
            return -1

        # Create the list of from and to currency value according to the selected date
        count = pos_begin
        if pos_begin != 1 and pos_end != -1:
            while count < pos_end:
                tmp_value_to = self.getCurrency(content, count, self.to_currency, currencies_list)
                tmp_value_from = self.getCurrency(content, count, self.from_currency, currencies_list)

                if tmp_value_from != 'N/A' and tmp_value_to != 'N/A':
                    self.list_to_values.append(float(tmp_value_to))
                    self.list_from_values.append(float(tmp_value_from))
                count = count + len(currencies_list)
        if len(self.list_from_values) == 0 or len(self.list_to_values) == 0:
            return -1
        return 0

    def getMaxRange(self, value_list):
        """

        :param value_list: The list which contains all the currencies values

        This function gets the maximum value in a list.

        :return: The maximum value in the list
        """
        count = 0
        for i in value_list:
            if i != 'N/A':
                if count < float(i):
                    count = float(i)
        return count

    def dograph(self):
        """
        This function print all values in the graph.
        :return: Nothing
        """

        self.rates_plot.clear()

        self.rates_plot.showGrid(x=True, y=True)
        if self.rates_plot.plotItem.legend:
            self.rates_plot.plotItem.legend.scene().removeItem(self.rates_plot.plotItem.legend)

        self.rates_plot.addLegend()
        self.rates_plot.setLabel('left', 'Rate')
        self.rates_plot.setLabel('bottom', 'Days')

        # set the range for the from and to values list
        rates_range_from = range(0, len(self.list_from_values))
        rates_range_to = range(0, len(self.list_to_values))

        # set the range of the conversion rates
        if len(self.list_from_values) > len(self.list_to_values):
            rates_range_convert = range(len(self.list_from_values))
        else:
            rates_range_convert = range(0, len(self.list_to_values))

        # Create conversion list and set each values to the amount
        count = 0
        conversion_list = []
        while count < len(rates_range_convert):
            if self.list_from_values[count] and self.list_to_values[count]:
                tmp_from = float(self.list_from_values[count])
                tmp_to =  float(self.list_to_values[count])
                tmp_from_amt = float(self.from_amount.value())
                tmp_amount = (tmp_from / tmp_to) * tmp_from_amt
                conversion_list.append(tmp_amount)
                count += 1
            else:
                break

        # Set the minimum and the maximum for the X and Y axis
        self.rates_plot.setXRange(0, len(self.list_from_values))
        max_to = self.getMaxRange(self.list_to_values)
        max_from = self.getMaxRange(self.list_from_values)
        if max_from > max_to:
            self.rates_plot.setYRange(0, max_from)
        else:
            self.rates_plot.setYRange(0, max_to)

        # Create the plot for from and to currencies and for the conversion rates
        self.c1 = self.rates_plot.plot(rates_range_from, self.list_from_values, pen='b', symbol='x', symbolPen='b', symbolBrush=0.2, name=self.from_currency)
        self.c2 = self.rates_plot.plot(rates_range_to, self.list_to_values, pen='r', symbol='o', symbolPen='r', symbolBrush=0.2, name=self.to_currency)
        self.c3 = self.rates_plot.plot(rates_range_convert, conversion_list, pen='g', symbol='+', symbolPen='g', symbolBrush=0.2, name='conversion rate')


app = QApplication(sys.argv)
form = Form()
form.show()
sys.exit(app.exec_())


# Screen Shots of working Project x 2
![Screen Shot1](./py3.png)
![Screen Shot1](./py4.png)
